In [0]:
from pyspark.sql import functions as F

(spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaLocation", "/Volumes/main/db_project/raw_data/_checkpoints/bronze_json_schema") 
    .load("/Volumes/main/db_project/raw_data/chunks/chunk3_json")
    .select(
        "*", 
        F.col("_metadata.file_path").alias("source"), # THIS IS KEY
        F.current_timestamp().alias("load_dt")
    ) 
    .writeStream
    .option("checkpointLocation", "/Volumes/main/db_project/raw_data/_checkpoints/bronze_json")
    .option("mergeSchema", "true") 
    .trigger(availableNow=True)
    .table("main.db_project.bronze_json_data")
)

In [0]:
%sql
-- This will show all columns that actually exist in the table
SELECT * FROM main.db_project.bronze_json_data LIMIT 5;

In [0]:
# This deletes the old progress markers so the stream starts fresh
#dbutils.fs.rm("/Volumes/main/db_project/raw_data/_checkpoints/bronze_json", True)
#dbutils.fs.rm("/Volumes/main/db_project/raw_data/_checkpoints/bronze_json_schema", True)